<a href="https://colab.research.google.com/github/mannb986/forest_cover_classification/blob/main/forest_cover_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forest Cover Classification Using Deep Learning

In this project, I will use deep learning to predict forest cover type (the most common kind of tree cover) based only on cartographic variables. The actual forest cover type for a given 30 x 30 meter cell was determined from US Forest Service (USFS) Region 2 Resource Information System data. The covertypes are the following:

* Spruce/Fir
* Lodgepole Pine
* Ponderosa Pine
* Cottonwood/Willow
* Aspen
* Douglas-fir
* Krummholz

Independent variables were then derived from data obtained from the US Geological Survey and USFS. It contains binary columns of data for qualitative independent variables such as wilderness areas and soil type.

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so existing forest cover types are mainly a result of ecological processes rather than forest management practices.

### Project Objectives:

* Develop one or more classifiers for this multi-class classification problem.
* Use TensorFlow with Keras to build the classifier(s).
* Hyperparameter tuning to improve performance. 
* Test & analyse performance. 
* Create clean and modular code. 

## Libraries

In [1]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
import numpy as np

## Loading Data

In [2]:
 from google.colab import files


uploaded = files.upload()

Saving cover_data.csv to cover_data.csv


In [3]:
data = pd.read_csv('cover_data.csv')

In [4]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [6]:
Counter(data['class'])

Counter({1: 211840, 2: 283301, 3: 35754, 4: 2747, 5: 9493, 6: 17367, 7: 20510})

In [7]:
y = data['class']

In [8]:
x = data.iloc[:, :-1]

In [9]:
x.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


## Data Processing



In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, stratify=y, test_size=0.33, random_state=42)

In [40]:
columns = X_train.columns

ct = ColumnTransformer([("numeric", StandardScaler(), columns)], remainder='passthrough')

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [41]:
Counter(Y_train)

Counter({1: 141933, 2: 189812, 3: 23955, 4: 1840, 5: 6360, 6: 11636, 7: 13742})

In [42]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

## Designing the Model

In [45]:
model = Sequential()

model.add(InputLayer(input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 64)                3520      
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_26 (Dense)             (None, 8)                 136       
Total params: 6,264
Trainable params: 6,264
Non-trainable params: 0
____________________________________________________

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train & Evaluate Model

In [47]:
from tensorflow.keras.callbacks import EarlyStopping

#stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4) 

history = model.fit(X_train, Y_train, epochs=10, batch_size=5, verbose=1, validation_split=0.1)

Epoch 1/10
70070/70070 [==============================] - 101s 1ms/step - loss: 0.6552 - accuracy: 0.7223 - val_loss: 0.5015 - val_accuracy: 0.7884
Epoch 2/10
70070/70070 [==============================] - 101s 1ms/step - loss: 0.5357 - accuracy: 0.7733 - val_loss: 0.4707 - val_accuracy: 0.8045
Epoch 3/10
70070/70070 [==============================] - 102s 1ms/step - loss: 0.5115 - accuracy: 0.7846 - val_loss: 0.4540 - val_accuracy: 0.8114
Epoch 4/10
70070/70070 [==============================] - 102s 1ms/step - loss: 0.5044 - accuracy: 0.7918 - val_loss: 0.4402 - val_accuracy: 0.8188
Epoch 5/10
70070/70070 [==============================] - 102s 1ms/step - loss: 0.4906 - accuracy: 0.7962 - val_loss: 0.4302 - val_accuracy: 0.8231
Epoch 6/10
70070/70070 [==============================] - 102s 1ms/step - loss: 0.4841 - accuracy: 0.7980 - val_loss: 0.4256 - val_accuracy: 0.8225
Epoch 7/10
70070/70070 [==============================] - 101s 1ms/step - loss: 0.4757 - accuracy: 0.8028 - val_

In [49]:
#get additional statistics
y_estimate = model.predict(X_test)
y_estimate = np.argmax(y_estimate, axis=1)
y_true = np.argmax(Y_test, axis=1)

#printing additional metrics such as F1 score
print(classification_report(y_true, y_estimate))

              precision    recall  f1-score   support

           1       0.83      0.84      0.83     69907
           2       0.84      0.88      0.86     93489
           3       0.79      0.80      0.80     11799
           4       0.70      0.57      0.63       907
           5       0.84      0.24      0.37      3133
           6       0.60      0.51      0.55      5731
           7       0.93      0.66      0.77      6768

    accuracy                           0.83    191734
   macro avg       0.79      0.64      0.69    191734
weighted avg       0.83      0.83      0.83    191734



## Results

Model has a F1 score of 0.83, there could be some further increases with changing the hyperparameters such as `epochs` which I will look to review another time. 